In [29]:
import pandas as pd
#need to add endoing='utf_8' for windows system
with open('cleaned_data_final.csv', 'r') as fp:
    df = pd.read_csv('cleaned_data_final.csv')

In [43]:
df = df.set_index('Object ID')
df = df[df['image_url'].notnull()]

KeyError: 'Object ID'

In [66]:
df['image_url']

Object ID
34        https://images.metmuseum.org/CRDImages/ad/orig...
37        https://images.metmuseum.org/CRDImages/ad/orig...
38        https://images.metmuseum.org/CRDImages/ad/orig...
39        https://images.metmuseum.org/CRDImages/ad/orig...
40        https://images.metmuseum.org/CRDImages/ad/orig...
41        https://images.metmuseum.org/CRDImages/ad/orig...
42        https://images.metmuseum.org/CRDImages/ad/orig...
43        https://images.metmuseum.org/CRDImages/ad/orig...
44        https://images.metmuseum.org/CRDImages/ad/orig...
45        https://images.metmuseum.org/CRDImages/ad/orig...
46        https://images.metmuseum.org/CRDImages/ad/orig...
47        https://images.metmuseum.org/CRDImages/ad/orig...
50        https://images.metmuseum.org/CRDImages/ad/orig...
51        https://images.metmuseum.org/CRDImages/ad/orig...
52        https://images.metmuseum.org/CRDImages/ad/orig...
53        https://images.metmuseum.org/CRDImages/ad/orig...
56        https://images.metmu

In [54]:
#store valid id list
object_id_list = list(df.index)

#Returns list of object IDs that have valid (non-NaN) value in that particular category column
#Category could be art title, artist name, country, time period, etc.
def get_valid_id_list(category):  
    import numpy as np
    
    valid_id_list = []
    valid_rows = np.where(df[category].notnull() & df[category].apply(lambda x: len(str(x))<50))[0].tolist()
    for x in valid_rows:
        valid_id_list.append(object_id_list[x])
        
    return valid_id_list

#create valid_id_dict of question categories
category_list = ['Artist Display Name','century','Title','Country','Department']
valid_id_dict = dict()
for x in category_list:
    valid_id_dict[x] = get_valid_id_list(x)

#randomly generate a category, object id
def get_question():
    import random
    category = random.choice(category_list)
    obj_id = random.choice(valid_id_dict[category])
    info = create_question(category, obj_id)
    pic_link = get_image(obj_id)
    title = get_title(obj_id)
    obj_link = get_url(obj_id)
    return info, pic_link, title, obj_link

#return the desired answer info of the randomly selected art
def get_image(ind):
    return df.loc[ind]['image_url']

def get_title(ind):
    return df.loc[ind]['Title']

def get_url(ind):
    return df.loc[ind]['Link Resource']

def get_artist(ind):
    return df.loc[ind]['Artist Display Name']

def get_time(ind):
    return df.loc[ind]['century']

def get_country(ind):
    return df.loc[ind]['Country']

def get_department(ind):
    return df.loc[ind]['Department']

#create choices for time questions
def gen_era_answers(century):
    import random
    interval=2 #the larger the interval, the easier; must be integers
    choices_u_bound=min(century+interval*random.randint(0,3),21)
    choice_list=[choices_u_bound,choices_u_bound-2,choices_u_bound-4,choices_u_bound-6]
    choices=dict()
    i=0
    for x in ['A','B','C','D']:
        choice=choice_list[i]
        if choice >0:
            choices[x]=f'A.D. {choice}th century'
        else:
            choice[x]=f'{choice}the century B.C.'
        i+=1
    if century >0:
        correct=f'A.D. {century}th century'
    else:
        correct=f'{century}th century B.C.'
    wrong_choices=[]
    for time in choices.values():
        if not time==correct:
            wrong_choices.append(time)
    return wrong_choices,correct

#generate question stem and answer choices
def create_question(category, obj_id):
    import random
    choice = list()
    obj_department = get_department(obj_id)
    if category is 'Artist Display Name':
        question = "Who is the artist for this piece of work?"
        correct = get_artist(obj_id)
        [wrong1,wrong2,wrong3] = generate_choices(correct,'Artist Display Name',obj_department)
    elif category is 'century':
        question = "During which time period did the creation of this work start?"
        wrong_answers,correct_answer=gen_era_answers(get_time(obj_id))
        correct=correct_answer
        wrong1 = wrong_answers[0]
        wrong2 = wrong_answers[1]
        wrong3 = wrong_answers[2]
    elif category is 'Country':
        question = "Where is this artwork from?"
        correct = get_country(obj_id)
        [wrong1,wrong2,wrong3] = generate_choices(correct,'Country',obj_department)
    else: #department
        question = "Which department does this exhibit belong to?"
        correct = get_department(obj_id)
        [wrong1,wrong2,wrong3] = generate_choices(correct,'Department',obj_department)
    choice = [wrong1, wrong2, wrong3, correct]
    return question, choice

#return 4 choices and answer key
def get_choices():
    import random
    from random import shuffle
    answer_list = get_question()[0][1]
    index_list = ['A','B','C','D']
    random.shuffle(answer_list)
    answer = dict(zip(index_list,answer_list))
    answer_key = [key for key, value in answer.items() if value == "correct"] #letter w/ correct answer
    answer_list=list(answer.items())
    answer_string_list=[]
    for answer in answer_list: #create list w/ entries like 'A. correct'
        x=answer[0]+'. '+answer[1]
        answer_string_list.append(x)
    return answer_key, answer_string_list

#return the list of valid artists and regions to be chosen from
def generate_candidates(df):
    department_list=list(df['Department'].unique())
    artist_candidates={}
    region_candidates={}
    for department in department_list:
        artist_candidates[department]=list(df[df['Department']==department]['Artist Display Name'].unique())
        artists=set(valid_id_dict['Artist Display Name']).intersection(set(artist_candidates))
        artist_candidates[department]=list(artists)
        region_candidates[department]=list(df[df['Department']==department]['Country'].unique())
        regions=set(valid_id_dict['Country']).intersection(set(region_candidates))
        region_candidates[department]=list(regions)
        
    return artist_candidates,region_candidates

#store them into df
artist_candidates,region_candidates=generate_candidates(df)
#return 3 wrong answers
def generate_choices(correct,category,obj_department):
    import random
    if category=='Artist Display Name':
        artists=[x for x in artist_candidates[obj_department] if not x==correct]
        wrong_choices=random.sample(artists,3)
    if category=='Country':
        regions=[x for x in region_candidates[obj_department] if not x==correct]
        wrong_choices=random.sample(regions,3)
    if category=='Department':
        department_list=list(df['Department'].unique())
        departments=[x for x in department_list if not x==correct]
        wrong_choices=random.sample(departments,3)
    return wrong_choices

In [77]:
from tkinter import *
from tkinter import scrolledtext
import urllib.request
from PIL import Image, ImageTk
import io
from io import BytesIO

#functions to be implemented: connect each page to the randomly generated question
#user can choose from a multiple choice question
#user is told right or wrong
#count the correct answer number
#quit button
class quizGUI:
    
    def __init__(self):
        root = Tk()
        root.lift()
        root.attributes("-topmost", True)
        root.geometry("700x600")
        root.title("MET-Q")
        self.root = root
        color = 'gainsboro'
        
        #canvas
        canvas_width = 700
        canvas_height = 600
        canvas = Canvas(root, height=canvas_height, width=canvas_width)
        self.canvas=canvas
        

        #correct/incorrect indicator
        var=StringVar()
        self.cor_label=Label(root,textvariable=var,height=2,width=8)
        self.cor_label.config(bg=color)
        self.var=var
        self.cor_label.grid(row=0,column=10,columnspan=2)
        
        #title label
        self.title_label = Label(root, text="The MET Quiz", font = "Times 20 bold")
        self.title_label.config(bg='red', fg='white')
        self.title_label.grid(row=0,column=1,columnspan=7)
    
        #record the number of correct answers so far
        self.total = IntVar() 
        self.total_var=StringVar()
        self.total_var.set('Number of correct answers: 0')
        self.score_label = Label(root,textvariable=self.total_var)
        self.score_label.config(bg=color)
        self.score_label.grid(row=1,column=2,columnspan=3)
        self.already_chosen=0
        
        #record the number of questions completed so far
        self.num_var=StringVar()
        self.num=IntVar()
        self.num_var.set('Questions completed: 0')
        self.num_of_ques=Label(root,textvariable=self.num_var)
        self.num_of_ques.config(bg=color)
        self.num_of_ques.grid(row=1,column=8,columnspan=3)
    
        #question stem
        question_var=StringVar()
        self.Q = Label(root, textvariable=question_var)
        question_var.set(get_question()[0][0])
        self.Q.grid(row=2,columnspan=9)
        self.Q.config(bg=color)
        self.question_var=question_var
        
        #object title
        obj_title_var = StringVar()
        self.obj_title = Label(root,textvariable=obj_title_var)
        obj_title_var.set(get_question()[2])
        self.obj_title.grid(row=3,columnspan=3)
        self.obj_title.config(bg=color)
        self.obj_title_var = obj_title_var
        
        #Question picture
#         URL=get_question()[1]
        #URL = 'https://bakingamoment.com/wp-content/uploads/2018/02/IMG_6207-chocolate-molten-lava-cakes-recipe-square.jpg'
        URL = 'https://images.metmuseum.org/CRDImages/ad/original/204788.jpg'
        with urllib.request.urlopen(URL) as url:
            s = url.read()
            image_file = Image.open(s)
#         u = urllib.request.urlopen(URL)
#         raw_data = u.read()
#         u.close()

#         image_file = Image.open(BytesIO(raw_data))
        image_file=image_file.resize((200,200),Image.ANTIALIAS)
        photo_image = ImageTk.PhotoImage(image_file)
        self.pic = Label(root,image=photo_image)
        self.pic.config(bg=color)
        self.pic.grid(row=4,columnspan=9)
        
        #choice buttons
        choices=get_choices()
        self.correct_answer=choices[0][0]
        self.choice_A=StringVar()
        self.choice_A.set(choices[1][0])
        self.choice_B=StringVar()
        self.choice_B.set(choices[1][1])
        self.choice_C=StringVar()
        self.choice_C.set(choices[1][2])
        self.choice_D=StringVar()
        self.choice_D.set(choices[1][3])
        self.A = Button(root, textvariable=self.choice_A,command=lambda: self.check_correct('A',self.correct_answer))
        self.A.grid(row=5,column=2,columnspan=5,pady=5)
        self.A.config(bg=color)
        self.B = Button(root, textvariable=self.choice_B,command=lambda: self.check_correct('B',self.correct_answer))
        self.B.grid(row=6,column=2,columnspan=5,pady=(0,5))
        self.B.config(bg=color)
        self.C = Button(root, textvariable=self.choice_C,command=lambda: self.check_correct('C',self.correct_answer))
        self.C.grid(row=7,column=2,columnspan=5,pady=(0,5))
        self.C.config(bg=color)
        self.D = Button(root, textvariable=self.choice_D,command=lambda: self.check_correct('D',self.correct_answer))
        self.D.grid(row=8,column=2,columnspan=5,pady=(0,5))
        self.D.config(bg=color)
        self.next=Button(root,text='Next question',command=lambda:self.next_()).grid(row=8,column=2,columnspan=5)
        self.next.config(bg=color)

        #quit button
        self.close_button=Button(root, text="Quit", command=lambda: root.destroy())
        self.close_button.grid(row=10,column=8)
        self.close_button.config(bg=color)
        
        #more information link
        self.more_info_link=get_question()[3]
        self.link=Button(root,text='Know more',command=lambda: self.open_page(self.more_info_link))
        self.link.grid(row=10,column=2,columnspan=5)
        self.link.config(bg=color)
        
        root.configure(background=color)
        root.mainloop() 

    #add 1 to score if the correct answer is chosen
    def check_correct(self,this_answer,correct_answer):
        if this_answer==correct_answer:
            self.correct()
        else:
            self.incorrect()
                        
    def correct(self):
        self.var.set('Correct')
        if self.already_chosen==0:
            self.total.set(self.total.get() + 1) 
            self.total_var.set('Number of correct answers: '+str(self.total.get()))
            self.already_chosen=1
    
    def incorrect(self):
        self.var.set('Incorrect')
        self.already_chosen=1
        
    def next_(self):
        next_question = get_question()
        question = next_question[0][0]
        next_pic = next_question[1]
        next_title = next_question[2]
        self.question_var.set(question)
        self.num.set(self.num.get()+1)
        self.num_var.set('Questions completed: '+str(self.num.get()))
        next_choices=get_choices()
        correctanswer=next_choices[0][0]
        self.correct_answer=correctanswer
        self.choice_A.set(next_choices[1][0])
        self.choice_B.set(next_choices[1][1])
        self.choice_C.set(next_choices[1][2])
        self.choice_D.set(next_choices[1][3])
        self.more_info_link = next_question[3]
        self.already_chosen=0
        
    def open_page(self,link):
        import webbrowser
        webbrowser.open(link)

In [78]:
myGUI = quizGUI()

FileNotFoundError: [Errno 2] No such file or directory: b'<html>\r\n<head>\r\n<META NAME="robots" CONTENT="noindex,nofollow">\r\n<script src="/_Incapsula_Resource?SWJIYLWA=5074a744e2e3d891814e9a2dace20bd4,719d34d31c8e3a6e6fffd425f7e032f3">\r\n</script>\r\n<body>\r\n</body></html>'